Median and Average Hourly Wages in the USA (1973-2022)


**Introduction:**
Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal

Clearly state the question you will try to answer with your project

Identify and describe the dataset that will be used to answer the question

**Preliminary exploratory data analysis:**

Demonstrate that the dataset can be read from the web into R 

Clean and wrangle your data into a tidy format

Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 

Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [6]:
library(tidyverse)

wage_data <- read_csv("https://raw.githubusercontent.com/asnbl/dsci-100-2023-24W-group-16/main/median_average_wages.csv")

wage_data <- wage_data |>
             select(year, white_average, black_average, hispanic_average)

tidy_wage_data <- pivot_longer(wage_data,
                               white_average:hispanic_average,
                               names_to = "Race",
                               values_to = "Wage") |>
                  mutate(Race = as_factor(Race))
tidy_wage_data

Rows: 50 Columns: 31
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (31): year, median, average, men_median, men_average, women_median, wome...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


year,Race,Wage
<dbl>,<fct>,<dbl>
2022,white_average,34.49
2022,black_average,25.61
2022,hispanic_average,24.84
2021,white_average,34.50
2021,black_average,25.40
2021,hispanic_average,24.90
2020,white_average,34.86
2020,black_average,26.03
2020,hispanic_average,25.29


**Methods:**

Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?

Describe at least one way that you will visualize the results


**Expected outcomes and significance:**

What do you expect to find?

What impact could such findings have?

What future questions could this lead to?